In [14]:
# Class label csv path
labels_csv_path = 'datasets/home_labels.csv'

In [15]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip
import sounddevice as sd
import numpy as np
from torch.nn import functional as F
from scipy.io.wavfile import write
import time
import threading
import queue
from IPython.display import display, clear_output

In [16]:
import serial

# ls /dev/tty.*
ser = serial.Serial('/dev/tty.usbmodem1201', 9600)

# while True:
#     if ser.in_waiting > 0:
#         line = ser.readline().decode('utf-8').strip()
#         print(line)

SerialException: [Errno 2] could not open port /dev/tty.usbmodem1201: [Errno 2] No such file or directory: '/dev/tty.usbmodem1201'

In [ ]:
# import csv

# label2id = {}
# id2label = {}
# with open(labels_csv_path, mode='r') as file:
#     csv_reader = csv.reader(file)
#     next(csv_reader)

#     for class_id, row in enumerate(csv_reader):
#         class_name = row[0]
#         label2id[class_name] = class_id
#         id2label[class_id] = class_name

# class_labels = list(label2id.keys())
# # class_labels.append("glass breaking")

# # print(f"{len(class_labels)} Unique classes: {class_labels}.")

In [22]:
import csv
csv_path='datasets/home_labels.csv'
label2id = {}
id2label = {}
with open(csv_path, mode='r') as file:
    csv_reader = csv.reader(file)

    for i, row in enumerate(csv_reader):
        class_name = row[0]
        label2id[class_name] = i
        id2label[i] = class_name

In [ ]:
import time
import joblib

window_size = 2  # seconds
sample_rate = 16000  # Hz

# Open the serial port
ser = serial.Serial('/dev/tty.usbmodem21201', 9600)

print("Loading model...")
# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version='2023', use_cuda=False)

print("Extracting text embeddings...")
# Extract text embeddings
text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}" for c in class_labels])

print("Listening...")

while True:    
    start_time = time.time()

    # Read a chunk of audio data from the serial port
    audio_data = ser.read(window_size * sample_rate * 2)  # 2 bytes per sample

    # Convert the audio data to an array of samples
    recording = np.frombuffer(audio_data, dtype=np.int16)

    # Save the recording to a temporary file
    temp_filename = "temp_recording.wav"
    write(temp_filename, sample_rate, recording)

    # Extract audio embeddings
    audio_embeddings = clap_model.get_audio_embeddings([temp_filename])
    # Compute similarity between audio and text embeddings
    similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

    similarity = F.softmax(similarities, dim=1)
    values, indices = similarity[0].topk(5)
    
    # # Check if "glass breaking" is in the top five predictions
    # things_to_detect = ["glass breaking", "using computer"]
    # detected = any(class_labels[index] in things_to_detect for index in indices)
    
    # print("CLAP predictions:\n" + "\n".join(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%" for value, index in zip(values, indices)) + "\n")
    
    detected = False
    clap_results = []
    print("\nCLAP predictions:")
    for value, index in zip(values, indices):
        index = index.item()
        value = round(value.item() * 100, 4)
        clap_results.append(index)
        clap_results.append(value)
        print(id2label[index], value)
        if id2label[index] in ["Crying", "Gunshot", "Glass breaking"] and value > 50: 
            print(f"ALERT: {id2label[index]} detected!")
            detected = True
            break
        
    # Ensemble
    if not detected:
        vclip_results = [-1, 0] * 5
        trained_ensemble = joblib.load('trained_RF_ensemble.joblib')
        X_test = np.expand_dims(np.hstack([vclip_results, clap_results]), axis=0)

        y_pred = trained_ensemble.predict(X_test)
        print("\nEmsemble prediction:", id2label[y_pred[0]])
        if id2label[y_pred[0]] in ["Crying", "Gunshot", "Glass breaking"]: 
            print(f"ALERT: {id2label[y_pred[0]]} detected!")


    os.remove(temp_filename) 
    # if detected:
    #     filename = f"recording_{int(time.time())}.wav"
    #     os.rename(temp_filename, filename)
    #     print(f"Alert! Glass breaking detected in {filename}")
    # else:
    #     os.remove(temp_filename) 

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Inference time is {elapsed_time} seconds.")


In [35]:
ser.close()